## Runnable Interface 介绍与使用

为了尽可能简化创建自定义链的过程，Langchain 实现了一个 **[Runnable](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable)** 协议。

许多 LangChain 组件都实现了 Runnable 协议，包括 chat models, LLMs, output parsers, retrievers, prompt templates等等。此外，还有几个用于处理可运行对象的[有用原语](https://python.langchain.com/v0.1/docs/expression_language/primitives/)。

Runnable 是一个标准接口，包括：

- stream：流式返回生成内容（chunk）
- invoke：对输入调用该链
- batch：对输入列表调用该链

不同组件的输入和输出类型有所差异:

| 组件    | 输入类型                                           | 输出类型           |
| ------------ | ----------------------------------------------------- | --------------------- |
| Prompt       | Dictionary                                            | PromptValue           |
| ChatModel    | Single string, list of chat messages or a PromptValue | ChatMessage           |
| LLM          | Single string, list of chat messages or a PromptValue | String                |
| OutputParser | The output of an LLM or ChatModel                     | Depends on the parser |
| Retriever    | Single string                                         | List of Documents     |
| Tool         | Single string or dictionary, depending on the tool    | Depends on the tool   |


所有 Runnable 对象都显式描述输入和输出 Schema，以检查输入和输出格式：

- input_schema：从Runnable的结构自动生成的输入Pydantic模型
- output_schema：从Runnable的结构自动生成的输出Pydantic模型 


### Input Schema

为了演示如何使用，下面我们创建一个超级简单的PromptTemplate + ChatModel链。

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_template("讲个关于 {topic} 的笑话吧")
chain = prompt | model

#### schema 方法

一个描述 Runnable 接受的输入的说明。这是根据任何 Runnable 结构动态生成的 Pydantic 模型。您可以调用 .schema() 来获取 `JSONSchema` 表示。

In [2]:
# 查看 Chain 的输入类型
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [3]:
# 查看 Prompt 的输入类型（Chain的输入从 Prompt 开始，因此输入类型一致）
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [4]:
# 查看 Model 的输入类型
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {

### Output Schema

输出类型仍然可以调用 .schema() 来获取其 `JSONSchema` 表示。

In [5]:
# 查看 Chain 的输出类型
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {'title': 'Id', 'type': 'string'}},
   'required': ['name', 'args', 'id']},
  'InvalidToolCall': {'title': 'InvalidToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'error': {'title': 'Error', 'type': 'string'}},
   'required': ['name', 'args', 'id', 'error']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'proper

### Stream

使用 .stream() 方法查看（同步）流式输出结果

In [8]:
for s in chain.stream({"topic": "程序员"}):
    print(s.content, end="", flush=True)

当然可以！这是一个关于程序员的笑话：

为什么程序员总是喜欢在黑暗中工作？

因为他们讨厌光明的错误（"light bugs"）！

希望这个笑话能让你笑一笑！

### Invoke
使用 .invoke() 方法单次（同步）调用

In [11]:
chain.invoke({"topic": "程序员"})

AIMessage(content='当然可以！这是一个关于程序员的笑话：\n\n有一天，一个程序员走进酒吧，点了一杯啤酒。酒保问他：“你喝酒吗？”程序员回答：“我不喝酒，我只喝代码！”\n\n酒保好奇地问：“那你怎么喝得下去呢？”\n\n程序员笑着说：“简单，我用错误调试器过滤掉所有的bug！”\n\n希望你喜欢这个笑话！', response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 18, 'total_tokens': 112}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-640a2b4f-66d7-495b-a165-e77eec0c47ba-0')

### Batch
使用 .batch() 方法（同步）批量调用

In [12]:
chain.batch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

[AIMessage(content='当然可以！这是一个关于程序员的笑话：\n\n有一天，程序员和他的朋友去钓鱼。朋友问：“你希望钓到什么鱼？”\n\n程序员回答：“我希望钓到一个‘bug’。”\n\n朋友疑惑地问：“为什么？”\n\n程序员笑着说：“因为我只要把它修复了，就能获得一个‘patch’！” \n\n希望你喜欢这个笑话！', response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 18, 'total_tokens': 107}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-3212674a-c3fc-43ed-92b9-4c58d810dc1e-0'),
 AIMessage(content='当然可以！这里有一个关于产品经理的笑话：\n\n有一天，一个产品经理走进一家咖啡店，点了一杯咖啡。店员问他：“要加糖吗？”\n\n产品经理回答：“我们能先进行一次用户调研吗？”\n\n店员愣了一下：“您是说，我先给您加糖，然后您再告诉我味道怎么样？”\n\n产品经理摇摇头：“不，我想要一个功能需求文档，说明加糖的利弊，以及用户反馈的分析。” \n\n店员无奈地说：“好的，那你等我写完这份文档，我再给你咖啡！”\n\n希望你喜欢这个笑话！', response_metadata={'token_usage': {'completion_tokens': 144, 'prompt_tokens': 16, 'total_tokens': 160}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-abf9b144-6b30-4588-9bf1-54c1ba9c596e-0'),
 AIMessage(content='当然可以！这是一个关于测试经理的笑话：\n\n有一天，一位测试经理走进

In [13]:
messages = chain.batch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

In [14]:
# 使用 StrOutputParser 来处理 Batch 批量输出
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

for idx, m in enumerate(messages):
    print(f"笑话{idx}:\n")
    print(output_parser.invoke(m))
    print("\n")

笑话0:

当然可以！这是一个关于程序员的笑话：

有一天，一个程序员去参加一个聚会。他的朋友问他：“你为什么总是那么忙，没时间出来玩？”

程序员回答：“我在修bug！”

朋友疑惑地问：“那你修好了吗？”

程序员叹了口气说：“修好了一个，又冒出来三个。” 

朋友调侃道：“那你还不如去玩，至少可以多交几个朋友！” 

程序员笑着说：“朋友太多也会有bug的！” 

希望这个笑话能让你开心！


笑话1:

当然可以！这里有一个关于产品经理的笑话：

一个产品经理走进咖啡店，看到菜单上有“黑咖啡”和“加奶咖啡”两种选择。他对店员说：“我想要一种新的咖啡，既有黑咖啡的浓郁，又有加奶咖啡的顺滑。”

店员想了想，回答：“那您要不要先试试黑咖啡，然后再加奶？”

产品经理微笑着说：“不，这样太慢了！我想要的是‘一键咖啡’！”

店员无奈地说：“好的，那您先付钱吧，‘一键咖啡’的支付方式还得等下个版本更新。”

希望这个笑话能让你开心！


笑话2:

当然可以！这是一个关于测试经理的笑话：

有一天，测试经理走进一个酒吧，点了一杯啤酒。酒保问他：“你要不要加点果汁？”

测试经理回答：“不，我只想要纯粹的啤酒！”

酒保又问：“那你要不要加点冰块？”

测试经理摇摇头：“不，我只想要没有问题的啤酒！”

酒保好奇地问：“那你怎么知道这杯啤酒没有问题？”

测试经理自信地说：“因为我已经提前测试过了它的味道！”

希望这个笑话能让你开心！




## 异步操作

这些方法也有相应的异步方法，应与 `asyncio` 的 `await` 语法一起使用以进行并发操作：

- astream：异步地流式返回生成内容（chunk）
- ainvoke：异步地对输入调用该链
- abatch：异步地对输入列表调用该链
- astream_log: 在发生时会返回中间步骤，并且最终返回结果之外。
- astream_events: beta 流式传输事件，在 langchain-core 0.1.14 中引入


### Async Stream

In [16]:
async for s in chain.astream({"topic": "狗"}):
    print(s.content, end="", flush=True)

当然可以！这是一个关于狗的笑话：

有一天，一只狗走进了酒吧，跳上吧台，对 bartender 说：“给我来一杯啤酒！”

bartender 惊讶地问：“你会说话？！”

狗回答：“当然！我还是个很聪明的狗！我能做很多事情！”

bartender 不敢相信，便问：“你能做什么呢？”

狗自豪地说：“我能算数学，还能写诗，甚至还能给你讲个笑话！”

bartender 兴奋地说：“好啊，那你给我讲个笑话！”

狗想了想，终于开口：“为什么狗总是喜欢追车？”

bartender 好奇地问：“为什么？”

狗笑着说：“因为它们想知道，车里的人到底在做什么，为什么他们总是比我跑得快！”

希望这个笑话能让你开心！

### Async Invoke

In [18]:
await chain.ainvoke({"topic": "程序员"})

AIMessage(content='当然可以！这是一个关于程序员的笑话：\n\n有一天，程序员的妻子对他抱怨：“你总是埋头工作，根本不关心我！”\n\n程序员回答：“我在调试啊，调试！”\n\n妻子好奇地问：“调试什么？”\n\n程序员认真地说：“调试我的感情代码！”\n\n希望你喜欢这个笑话！', response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 18, 'total_tokens': 103}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-29bed505-4809-44b7-a2d4-113d9a4347e0-0')

### Async Batch

In [19]:
await chain.abatch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

[AIMessage(content='当然可以！这里有一个关于程序员的笑话：\n\n有一天，一个程序员走进酒吧，点了一杯啤酒。喝完后，他发现自己忘记了钱包，于是他就问酒保：“我可以用代码来支付吗？”\n\n酒保愣了一下，回答：“当然不行！你不能用代码来支付账单！”\n\n程序员撇了撇嘴：“那我可以用‘调试’来支付吗？”\n\n酒保好奇地问：“调试是什么？”\n\n程序员微笑着说：“就是把错误都修复了，再让你给我打折！”\n\n希望你喜欢这个笑话！', response_metadata={'token_usage': {'completion_tokens': 139, 'prompt_tokens': 18, 'total_tokens': 157}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-18750583-14a1-44b7-af56-ea142b0970f1-0'),
 AIMessage(content='当然可以！这是一个关于产品经理的笑话：\n\n有一天，一个产品经理走进了一家咖啡店，点了一杯咖啡。咖啡师问他：“您要什么样的咖啡？”\n\n产品经理想了想，回答说：“我要一杯用户反馈最好的咖啡，最好再加一点创新和灵活性！”\n\n咖啡师愣了一下，问：“那您要加糖吗？”\n\n产品经理说：“糖的添加要根据用户需求再决定，不过我希望能有一个AB测试来验证一下！”\n\n咖啡师无奈地说：“好吧，那我先给您来一杯水吧，等您做完测试再决定要不要加糖！”\n\n希望这个笑话能让你开心！', response_metadata={'token_usage': {'completion_tokens': 165, 'prompt_tokens': 16, 'total_tokens': 181}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_9722793223', 'finish_reason': 'stop', 'logprobs': None}, id='run-6a8729a7-71a9-40b4-